How to create a Deployment
==========================

In this notebook, we show you how to create a Deployment with 3 ReplicaSets. These ReplicaSets are owned by the Deployment and are managed by the Deployment controller. We would also learn how to carry out RollingUpdate and RollBack to new and older versions of the deployment.

In [ ]:
%pip install kubernetes

In [ ]:
from kubernetes import client, config

### Load config from default location

In [55]:
config.load_kube_config()
apps_api = client.AppsV1Api()

### Create Deployment object

In [85]:
deployment = client.V1Deployment()

### Fill required Deployment fields (apiVersion, kind, and metadata)

In [ ]:
deployment.metadata = client.V1ObjectMeta(name="nginx-deployment")

### A Deployment also needs a .spec section

### Add Pod template in .spec.template section

### Pod template container description

In [87]:
spec_template_spec_container = client.V1Container(name="nginx")
spec_template_spec_container.name="nginx"
spec_template_spec_container.image="nginx:1.27.4"
spec_template_spec_container.ports = [client.V1ContainerPort(container_port=80)]

In [ ]:
spec_template = client.V1PodTemplateSpec()
spec_template.metadata = client.V1ObjectMeta(labels={"app": "nginx"})
spec_template.spec = client.V1PodSpec(containers=[spec_template_spec_container])

In [ ]:
spec = client.V1DeploymentSpec(template=spec_template, selector=client.V1LabelSelector(match_labels={"app": "nginx"}))

spec.replicas = 3

deployment.spec = spec

print(deployment)

{'api_version': None,
 'kind': None,
 'metadata': {'annotations': None,
              'creation_timestamp': None,
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': None,
              'labels': None,
              'managed_fields': None,
              'name': 'nginx-deployment',
              'namespace': None,
              'owner_references': None,
              'resource_version': None,
              'self_link': None,
              'uid': None},
 'spec': {'min_ready_seconds': None,
          'paused': None,
          'progress_deadline_seconds': None,
          'replicas': 3,
          'revision_history_limit': None,
          'selector': {'match_expressions': None,
                       'match_labels': {'app': 'nginx'}},
          'strategy': None,
          'template': {'metadata': {'annotations': None,
                                  

### Create Deployment

In [ ]:
apps_api.create_namespaced_deployment(namespace="default", body=deployment)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2025, 4, 3, 19, 58, 39, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 1,
              'labels': None,
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revisionHistoryLimit': {},
                                                           'f:selector': {},
                                                           'f:strategy': {'f:rollingUpdate': {'.': {},
                                  

### Update container image 

In [103]:
deployment.spec.template.spec.containers[0].image = "nginx:1.24.0"

### Apply update (RollingUpdate)

In [104]:
apps_api.replace_namespaced_deployment(name="nginx-deployment", namespace="default", body=deployment)

{'api_version': 'apps/v1',
 'kind': 'Deployment',
 'metadata': {'annotations': None,
              'creation_timestamp': datetime.datetime(2025, 4, 3, 19, 58, 39, tzinfo=tzutc()),
              'deletion_grace_period_seconds': None,
              'deletion_timestamp': None,
              'finalizers': None,
              'generate_name': None,
              'generation': 2,
              'labels': None,
              'managed_fields': [{'api_version': 'apps/v1',
                                  'fields_type': 'FieldsV1',
                                  'fields_v1': {'f:spec': {'f:progressDeadlineSeconds': {},
                                                           'f:replicas': {},
                                                           'f:revisionHistoryLimit': {},
                                                           'f:selector': {},
                                                           'f:strategy': {'f:rollingUpdate': {'.': {},
                                  

### Delete Deployment

In [100]:
apps_api.delete_namespaced_deployment(name="nginx-deployment", namespace="default", body=client.V1DeleteOptions(propagation_policy="Foreground", grace_period_seconds=5))

{'api_version': 'apps/v1',
 'code': None,
 'details': None,
 'kind': 'Deployment',
 'message': None,
 'metadata': {'_continue': None,
              'remaining_item_count': None,
              'resource_version': '17989',
              'self_link': None},
 'reason': None,
 'status': "{'observedGeneration': 3, 'replicas': 3, 'updatedReplicas': 3, "
           "'readyReplicas': 3, 'availableReplicas': 3, 'conditions': "
           "[{'type': 'Available', 'status': 'True', 'lastUpdateTime': "
           "'2025-04-03T19:56:41Z', 'lastTransitionTime': "
           "'2025-04-03T19:56:41Z', 'reason': 'MinimumReplicasAvailable', "
           "'message': 'Deployment has minimum availability.'}, {'type': "
           "'Progressing', 'status': 'True', 'lastUpdateTime': "
           "'2025-04-03T19:57:46Z', 'lastTransitionTime': "
           "'2025-04-03T19:56:34Z', 'reason': 'NewReplicaSetAvailable', "
           '\'message\': \'ReplicaSet "nginx-deployment-7f9d555b89" has '
           "successful

### Deploy a Deployment YAML manifest file

In [130]:
import yaml

with open(file="deployment.yaml") as f:

    dep = yaml.safe_load(f)

    k8s_apps_v1 = client.AppsV1Api()
    
    response = k8s_apps_v1.create_namespaced_deployment(body=dep, namespace="webapp")
    
    print(f"Deployment created. Status='{response.metadata.name}'")

Deployment created. Status='inspectorgadget'


### Deploy a Namespace YAML manifest file

In [ ]:
with open(file="namespace.yaml") as f:

    namespace = yaml.safe_load(f)

    core_v1 = client.CoreV1Api()

    response = core_v1.create_namespace(body=namespace)

    print(f"Namespace created. Status='{response.metadata.name}'")

Namespace created. Status='webapp'


### Deploy a Service YAML manifest file

In [131]:
with open(file="service.yaml") as f:

    serviceYaml = yaml.safe_load(f)

    service = client.CoreV1Api()

    response = service.create_namespaced_service(body=serviceYaml, namespace="webapp")

    print(f"Namespace created. Status='{response.metadata.name}'")

ApiException: (409)
Reason: Conflict
HTTP response headers: HTTPHeaderDict({'Audit-Id': '97e73e33-b23a-4405-be71-b8066dc3b529', 'Cache-Control': 'no-cache, private', 'Content-Type': 'application/json', 'X-Kubernetes-Pf-Flowschema-Uid': '74cd85a7-68b9-4097-b23d-23e5f4e08691', 'X-Kubernetes-Pf-Prioritylevel-Uid': '2a8278ac-fef7-4439-ba82-b6b9f3a7c7bc', 'Date': 'Thu, 03 Apr 2025 20:32:34 GMT', 'Content-Length': '216'})
HTTP response body: {"kind":"Status","apiVersion":"v1","metadata":{},"status":"Failure","message":"services \"inspectorgadget\" already exists","reason":"AlreadyExists","details":{"name":"inspectorgadget","kind":"services"},"code":409}

